In [ ]:
%matplotlib inline
"""
Golowasch, J., Casey, M., Abbott, L. F., & Marder, E. (1999).
Network Stability from Activity-Dependent Regulation of Neuronal Conductances.
Neural Computation, 11(5), 1079-1096.
https://doi.org/10.1162/089976699300016359
"""
from brian2.only import *
import brian2.numpy_ as np
import matplotlib.pyplot as plt

#set_device('cpp_standalone', build_on_run=False, directory=None)
seed(123456)
defaultclock.dt = 0.01*ms

init_time = 2.5*second
observe_time = 4*second
adapt_time = 49 * second
Delta_T = 17.5*mV
v_T = -40*mV
tau = 2*ms
tau_adapt = .02*second
tau_Ca = 150*ms
tau_x = 2*second
v_r = -68*mV
a = 1/Delta_T**3
b = 3/Delta_T**2
d = 2.5*nA/Delta_T**2
C = 60*pF
S = 2*nA/Delta_T
G = 28.5*nS
tau_z = 5*second
c = 1.2*nA
eqs = '''
dv/dt = (Delta_T*g*(-a*(v - v_T)**3 + b*(v - v_T)**2) + w - x - I_fast - I_slow)/C : volt
dw/dt = (c - d*(v - v_T)**2 - w)/tau : amp
dx/dt = (s*(v - v_r) - x)/tau_x : amp
s = S*(1 - tanh(z)) : siemens
g = G*(1 + tanh(z)) : siemens
dCa/dt = -Ca/tau_Ca : 1
dz/dt = tanh(Ca - Ca_target)/tau_z : 1
I_fast : amp
I_slow : amp
Ca_target : 1 (constant)
label : integer (constant)
'''
ABPD, LP, PY = 0, 1, 2

circuit = NeuronGroup(3, eqs, threshold='v>-20*mV', refractory='v>-20*mV', method='rk2',
                      reset='Ca += 0.1')
circuit.label = [ABPD, LP, PY]

# circuit.g = 28.5*nS  #'30*nS*(rand() + 0.5)'
circuit.v = v_r
circuit.w = '-5*nA*rand()'
circuit.z = 'rand()*0.2 - 0.1'
circuit.Ca_target = np.array([0.048, 0.0384, 0.06])

# Synapses
s_fast = 0.2/mV
V_fast = -50*mV
s_slow = 1/mV
V_slow = -55*mV
E_syn = -75*mV
k_1 = 1/ms
eqs_fast = '''
g_fast : siemens (constant)
I_fast_post = g_fast*(v_post - E_syn)/(1+exp(s_fast*(V_fast-v_pre))) : amp (summed)
'''
fast_synapses = Synapses(circuit, circuit, model=eqs_fast)
fast_synapses.connect('label_pre != label_post and not (label_pre == PY and label_post == ABPD)')
fast_synapses.g_fast['label_pre == ABPD and label_post == LP'] = 0.015*uS
fast_synapses.g_fast['label_pre == ABPD and label_post == PY'] = 0.005*uS
fast_synapses.g_fast['label_pre == LP and label_post == ABPD'] = 0.01*uS
fast_synapses.g_fast['label_pre == LP and label_post == PY']   = 0.02*uS
fast_synapses.g_fast['label_pre == PY and label_post == LP']   = 0.005*uS

eqs_slow = '''
k_2 : 1/second (constant)
g_slow : siemens (constant)
I_slow_post = g_slow*m_slow*(v_post-E_syn) : amp (summed)
dm_slow/dt = k_1*(1-m_slow)/(1+exp(s_slow*(V_slow-v_pre))) - k_2*m_slow : 1 (clock-driven)
'''
slow_synapses = Synapses(circuit, circuit, model=eqs_slow, method='exact')
slow_synapses.connect('label_pre == ABPD and label_post != ABPD')
slow_synapses.g_slow['label_post == LP'] = 0.025*uS
slow_synapses.k_2['label_post == LP']    = 0.03/ms
slow_synapses.g_slow['label_post == PY'] = 0.015*uS
slow_synapses.k_2['label_post == PY']    = 0.008/ms

M = StateMonitor(circuit, ['v'], record=True, dt=.1*ms)
spikes = SpikeMonitor(circuit)

M.active = False
run(init_time, report='text')
M.active = True
run(observe_time, report='text')
M.active = False
run(adapt_time, report='text')
M.active = True
run(observe_time, report='text')
#device.build()

spike_trains = spikes.spike_trains()

In [ ]:
def do_pyloric_net_plot(spike_trains, times, membrane_potential, varname,
                        filename, init_time, observe_time, adapt_time):
    try:
        plt.style.use('paper_mplstyle.cfg')
    except IOError:
        pass
    fig, axes = plt.subplots(4, 2, sharex='col', sharey='row',
                             gridspec_kw={'top': 0.86, 'left': 0.275,
                                          'bottom': 0.25, 'right': 0.95,
                                          'height_ratios': [2, 2, 2, 1]},
                             figsize=(5, 5), dpi=100)
    for idx, (label, color) in enumerate(zip(['AB/PD', 'LP', 'PY'],
                                             ['C0', 'C1', 'C2'])):
        axes[idx, 0].plot((times - init_time) / second, membrane_potential[idx] / mV, label=label,
                          color=color)
        if spike_trains is not None:
            axes[3, 0].vlines((spike_trains[idx] - init_time) / second,
                              np.ones(len(spike_trains[idx]))*(3-idx)-0.5, np.ones(len(spike_trains[idx]))*(3-idx)+0.5,
                              color=color)
        after_adapt_time = init_time + observe_time + adapt_time
        axes[idx, 1].plot((times - after_adapt_time) / second, membrane_potential[idx] / mV, label=label,
                          color=color)
        if spike_trains is not None:
            axes[3, 1].vlines((spike_trains[idx] - after_adapt_time) / second,
                              np.ones(len(spike_trains[idx]))*(3-idx)-0.5, np.ones(len(spike_trains[idx]))*(3-idx)+0.5,
                              color=color)

    axes[0, 0].set(xlim=(0, observe_time/second), title='initial')
    axes[0, 1].set(xlim=(0, observe_time/second), title='adapted')
    axes[3, 0].set_yticks([])
    axes[3, 0].spines['left'].set_visible(False)
    axes[3, 1].spines['left'].set_visible(False)
    axes[2, 0].set(xlabel='time (in s)', ylabel='${}$ (in mV)'.format(varname))
    axes[2, 0].yaxis.set_label_coords(-0.6, 1.35)
    axes[2, 0].xaxis.set_label_coords(1.17, -1.5)
    if filename is not None:
        fig.savefig(filename, transparent=True)
    else:
        plt.show()

do_pyloric_net_plot(spike_trains, M.t, M.v, 'v', None,
                    init_time, observe_time, adapt_time)